In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np

from aml_storage import Labels, Block, Descriptor

from utils.librascal import RascalSphericalExpansion

import ase.io

In [3]:
frames = ase.io.read("data/molecule_conformers_dftb.xyz", ":2")

In [4]:
rascal_hypers = {
    "interaction_cutoff": 3.5,
    "cutoff_smooth_width": 0.5,
    "max_radial": 6,
    "max_angular": 6,
    "gaussian_sigma_type": "Constant",
    "compute_gradients": True,
}

calculator = RascalSphericalExpansion(rascal_hypers)
descriptor = calculator.compute(frames)

/Users/guillaume/code/aml-storage-examples/utils/librascal.py:92: UserWarning: numpy/forward gradients are currently broken with librascal,please use rascaline instead
  warnings.warn(


In [5]:
# A descriptor contains a set of sparse labels, labeling the different blocks in
# the descriptor

print(descriptor.sparse.names)
print(descriptor.sparse[:30:5])

('spherical_harmonics_l', 'center_species', 'neighbor_species')
[(0, 1, 1) (0, 6, 8) (1, 1, 6) (1, 8, 1) (2, 1, 8) (2, 8, 6)]


In [6]:
# we can also represent labels as named tuples
for i, label in enumerate(descriptor.sparse.as_namedtuples()):
    if i > 3:
        break
    print(label)
    print(label.as_dict())
    print()

LabelTuple(spherical_harmonics_l=0, center_species=1, neighbor_species=1)
{'spherical_harmonics_l': 0, 'center_species': 1, 'neighbor_species': 1}

LabelTuple(spherical_harmonics_l=0, center_species=1, neighbor_species=6)
{'spherical_harmonics_l': 0, 'center_species': 1, 'neighbor_species': 6}

LabelTuple(spherical_harmonics_l=0, center_species=1, neighbor_species=8)
{'spherical_harmonics_l': 0, 'center_species': 1, 'neighbor_species': 8}

LabelTuple(spherical_harmonics_l=0, center_species=6, neighbor_species=1)
{'spherical_harmonics_l': 0, 'center_species': 6, 'neighbor_species': 1}



In [7]:
# These labels can then be used to access different blocks
block = descriptor.block(
    spherical_harmonics_l=4, 
    center_species=1, 
    neighbor_species=1,
)

# a block contains a `values` array, the shape of which is determined by 
# three other set of labels: samples, components, and features
print(block.values.shape)

(12, 9, 6)


In [8]:
# The samples labels indicate **what** we are representing
print(block.samples.names)
print(block.samples[:6])

('structure', 'center')
[(0, 4) (0, 5) (0, 6) (0, 7) (0, 8) (0, 9)]


In [9]:
# The components labels are optional and indicate components of tensorial/vector quantities
print(block.components.names)
print(block.components)

('spherical_harmonics_m',)
[(-4,) (-3,) (-2,) (-1,) ( 0,) ( 1,) ( 2,) ( 3,) ( 4,)]


In [10]:
# The feature labels indicate **how** we are representing the samples, here we
# are using a radial basis indexed by `n`
print(block.features.names)
print(block.features)

('n',)
[(0,) (1,) (2,) (3,) (4,) (5,)]


In [11]:
# the block can also contain gradients of the values w.r.t. different variables, 
# the most commong being the positions of the atoms in the system
gradients_samples, gradients = block.gradient("positions")

# the gradients have their own set of sample labels, while the symmetric and
# feature labels are shared with the values
print(gradients.shape)

# the gradients samples indicate which value `sample` is being considered, and 
# with respect to which atom/spatial direction the gradients are being taken 
print(gradients_samples.names)
print(gradients_samples[:10])

(156, 9, 6)
('sample', 'structure', 'atom', 'spatial')
[(0, 0, 4, 0) (0, 0, 4, 1) (0, 0, 4, 2) (0, 0, 9, 0) (0, 0, 9, 1)
 (0, 0, 9, 2) (0, 0, 6, 0) (0, 0, 6, 1) (0, 0, 6, 2) (0, 0, 5, 0)]


In [12]:
# since there is a single oxygen atom, there are no contribution to the gradient
# with center_specie=8, neighbor_species=8, spherical_harmonics_l>0
block = descriptor.block(
    spherical_harmonics_l=4, 
    center_species=8, 
    neighbor_species=8,
)

gradients_samples, gradients = block.gradient("positions")
assert len(gradients) == 0

# Moving labels around (from sparse to dense storage)

In [13]:
rascal_hypers["compute_gradients"] = False

calculator = RascalSphericalExpansion(rascal_hypers)
descriptor = calculator.compute(frames)

In [14]:
# we can group together multiple block by moving a label from sparse to the features.

descriptor.sparse_to_features("neighbor_species")

In [15]:
block = descriptor.block(center_species=1, spherical_harmonics_l=6)

print(block.values.shape)
print(block.samples.names)
print(block.samples)

(12, 13, 18)
('structure', 'center')
[(0, 4) (0, 5) (0, 6) (0, 7) (0, 8) (0, 9) (1, 4) (1, 5) (1, 6) (1, 7)
 (1, 8) (1, 9)]


In [16]:
# depending on the blocks, we might not be able to move all sparse labels to
# features
try:
    descriptor.sparse_to_features("spherical_harmonics_l")
except Exception as e:
    print(e)

invalid parameter: can not move sparse label to features if the blocks have different components labels, call components_to_features first


In [17]:
# we need to move the m index to features before moving l to features
descriptor.components_to_features()

descriptor.sparse_to_features("spherical_harmonics_l")

In [18]:
block = descriptor.block(center_species=1)
block.values.shape

(12, 1, 882)

In [19]:
descriptor.sparse_to_samples("center_species")

In [20]:
# we now only have one block, containing everything
block = descriptor.block()
block.values.shape

(20, 1, 882)

## Checking vs librascal storage

In [21]:
calculator = RascalSphericalExpansion(rascal_hypers)
descriptor = calculator.compute(frames)

descriptor.sparse_to_features("neighbor_species")
species_radial_size = descriptor.block(spherical_harmonics_l=0, center_species=1).values.shape[2]

descriptor.components_to_features()
descriptor.sparse_to_features("spherical_harmonics_l")
descriptor.sparse_to_samples("center_species")

block = descriptor.block()
n_features = block.features.shape[0]

full_dense = block.values

# put lm at the end of the features
full_dense = full_dense.reshape(full_dense.shape[0], -1, species_radial_size)
full_dense = full_dense.swapaxes(1, 2)
full_dense = full_dense.reshape(full_dense.shape[0], -1)

In [22]:
from rascal.representations import SphericalExpansion

rascal_calculator = SphericalExpansion(**rascal_hypers)
managers = rascal_calculator.transform(frames)
rascal_spx = managers.get_features(rascal_calculator)

In [23]:
assert np.all(rascal_spx == full_dense)